In [37]:
import numpy as np
import pandas as pd
import sys 
eps = np.finfo(float).eps
from numpy import log2 as log

In [38]:
attributes = {
    'A1': ['b','a'],
    'A2': 'continuous',
    'A3': 'continuous',
    'A4': ['u','y','l','t'],
    'A5': ['g','p','gg'],
    'A6': ['c','d','cc','i','j','k','m','r','q','w','x','e','aa','ff'],
    'A7': ['v','h','bb','j','n','z','dd','ff','o'],
    'A8': 'continuous',
    'A9': ['t','f'],
    'A10': ['t','f'],
    'A11': 'continuous',
    'A12': ['t','f'],
    'A13': ['g','p','s'],
    'A14': 'continuous',
    'A15': 'continuous',
    'A16': ['+','-'],
}

In [39]:
train_df= pd.read_csv("train.txt",sep = '\t', header = None, names = attributes.keys())
train_df
# type(df)

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202,0,+
1,a,24.5,0.500,u,g,q,h,1.50,t,f,0,f,g,280,824,+
2,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120,0,+
3,b,32.08,4.000,u,g,m,v,2.50,t,f,0,t,g,360,0,+
4,a,45.83,10.500,u,g,q,v,5.00,t,t,7,t,g,0,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,260,0,-
486,a,22.67,0.750,u,g,c,v,2.00,f,t,2,t,g,200,394,-
487,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,t,g,200,1,-
488,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,280,750,-


In [40]:
def fill_missing_values(df):
    for key in df.keys()[:-1]:
        df[key] = df[key].replace('?',df[key].mode()[0])
        
fill_missing_values(train_df)

In [41]:
threshold_dict = {}

In [42]:
# entropy_node = 0  #Initialize Entropy
# values = df.A16.unique()  #Unique objects - 'Yes', 'No'
# for value in values:
#     fraction = df.A16.value_counts()[value]/len(df.A16)  
#     entropy_node += -fraction*np.log2(fraction)

# print(entropy_node)

def find_entropy(df):
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy

def entropy_continuous(df,data_portion):
    data_portion_count = len(data_portion)
#     print("In entropy continuous:" , data_portion_count)
    output = {}
    entropy = 0
    for x,y in data_portion:
        if y in output:
            output[y].append((x,y))
        else:
            output[y] = [(x,y)]
#     print("In entropy continuous:", output)
#     print("In entropy continuous:", len(output.get('+')))
    if ('+' in output.keys()):
        pos_labels_count = len(output.get('+'))
    else:
        pos_labels_count = 0
    neg_labels_count = data_portion_count - pos_labels_count
    pos_fraction = pos_labels_count/(data_portion_count+eps)
    pos_entropy = -pos_fraction*log(pos_fraction+eps)
    neg_fraction = neg_labels_count/(data_portion_count+eps)
    neg_entropy = -neg_fraction*log(neg_fraction+eps)
    entropy = pos_entropy + neg_entropy
    return entropy

def find_entropy_attribute(df,key):
    if(type(attributes.get(key))==list):
        Class = df.keys()[-1]   #To make the code generic, changing target variable class name
        target_variables = df[Class].unique()  #This gives all 'Yes' and 'No'
        variables = df[key].unique()    #This gives different features in that attribute (like 'Hot','Cold' in Temperature)
        entropy2 = 0
        threshold = None
        for variable in variables:
            entropy = 0
            for target_variable in target_variables:
                num = len(df[key][df[key]==variable][df[Class] == target_variable])
                den = len(df[key][df[key]==variable])
                fraction = num/(den+eps)
                entropy += -fraction*log(fraction+eps)
            fraction2 = den/len(df)
            entropy2 += fraction2*entropy
#             entropy2 += -fraction2*entropy
    else:
        threshold = 0
        min_entropy = sys.maxsize
        entropy2 = 0
        threshold = None
        zipped = list(zip(df[key], df[df.keys()[-1]]))
        result = sorted(zipped, key = lambda x: x[0])
#         print(result)
        for i in range(len(result)):
            entropy2 = 0
            if(i!=0 and result[i][0]!=result[i-1][0]): #check for all possible cases
                mid_value = (float(result[i][0])+float(result[i-1][0]))/2.0
#                 print(mid_value)
                left_part = result[0:i]
                left_part_count = len(left_part)
                if(left_part_count == 0):
                    continue
#                 print(left_part)
                entropy_left = entropy_continuous(df,left_part)
                left_fraction = len(left_part)/len(df)
                weighted_entropy_left = left_fraction*entropy_left
                entropy2+= weighted_entropy_left
#                 print(left_part_count)
                right_part = result[i:]
                right_part_count = len(right_part)
                if(right_part_count != 0):
                    entropy_right = entropy_continuous(df,right_part)
                    right_fraction = len(right_part)/len(df)
                    weighted_entropy_right = right_fraction*entropy_right
                    entropy2+= weighted_entropy_right
                    if(entropy2 < min_entropy):
                        min_entropy = entropy2
                        threshold = mid_value
        entropy2 = min_entropy
        threshold_dict[key] = threshold
        #         print(right_part_count)
                
#             else:
#                 print("Do nothing:", i)
    return (entropy2,threshold)
#     return abs(entropy2)
    

def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
#         Entropy_att.append(find_entropy_attribute(df,key))
        entropy_attribute, threshold = find_entropy_attribute(df, key)
        IG.append(find_entropy(df)-entropy_attribute)
    return df.keys()[:-1][np.argmax(IG)], threshold

# entropy_target = find_entropy(df)-find_entropy_attribute(df,'A6')
# entropy_target

In [44]:
int(find_winner(train_df)[0][1:])
entropy_attribute, threshold = find_winner(train_df)
print(entropy_attribute, threshold)
# threshold_dict

A9 241.0


In [128]:
df['A16'].describe()

count     490
unique      2
top         -
freq      277
Name: A16, dtype: object

In [9]:
def check_purity(data):
    label_column = data.keys()[-1]
    unique_classes = np.unique(data[label_column])

    if len(unique_classes) == 1:
        return True
    else:
        return False

In [10]:
check_purity(train_df)

False

In [12]:
def majority_class(data):
    
    label_column = data.keys()[-1]
    unique_classes, counts_unique_classes = np.unique(data[label_column], return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

In [13]:
majority_class(train_df)

'-'

In [26]:
def determine_type_of_feature(df):
    feature_types = []
    n_unique_values_treshold = 14
    for feature in df.columns:
#         print(feature)
        if feature != "A16":
            unique_values = df[feature].unique()
            example_value = unique_values[0]

            if (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
#                 print('c')
            else:
                feature_types.append("continuous")
#                 print('conti')
    
    return feature_types
feature_types = determine_type_of_feature(train_df)
feature_types[int(find_winner(train_df)[0][1:])]

'categorical'

In [2]:
def convert_continuous_to_categorical(df,threshold_dict):
    for key in threshold_dict:
        print("Key value:", key)
        df[key].astype(np.float)
        df.loc[df[key] > threshold_dict.get(key), key] = 1
        df.loc[df[key] <= threshold_dict.get(key), key] = 0
#         df[key] = np.where(df[key] >= threshold_dict.get(key), 1, df[key])
#         df[key] = np.where(df[key] < threshold_dict.get(key), 0, df[key])
        print("Done")
convert_continuous_to_categorical(df,threshold_dict)
# df['A2'].values
# type(threshold_dict.get('A2'))
# df['A14'] = df['A14'].astype(np.float)
# type(df['A2'])
# df['A14'].values
# threshold_dict.values()
# df

NameError: name 'df' is not defined

In [14]:
entropy = find_entropy(train_df)
entropy

0.987658900015369

In [15]:
find_entropy_attribute(train_df,'A8')

(0.8600634346339666, 1.1875)

In [17]:
def get_subtable(df, node,value):
    return df[df[node] == value].reset_index(drop=True)


def buildTree(df,tree=None): 
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    
    #Here we build our decision tree

    #Get attribute with maximum information gain
    node, threshold = find_winner(df)
    
    #Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
    
    attValue = np.unique(df[node])
    
    #Create an empty dictionary to create tree    
    if tree is None:                    
        tree={}
        tree[node] = {}
    
   #We make loop to construct a tree by calling this function recursively. 
    #In this we check if the subset is pure and stops if it is pure. 

    for value in attValue:
        
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable[Class],return_counts=True)                        
        
        if len(counts)==1:#Checking purity of subset
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable) #Calling the function recursively 
                   
    return tree

In [18]:
import pprint

tree = buildTree(df)
pprint.pprint(tree)

{'A9': {'f': {'A6': {'aa': '-',
                     'c': {'A3': {0.0: {'A2': {'20.42': '-',
                                               '21.17': '-',
                                               '25.58': '+',
                                               '34.58': '-',
                                               '37.58': '+'}},
                                  0.125: '-',
                                  0.165: '+',
                                  0.29: '-',
                                  0.375: '-',
                                  0.415: '-',
                                  0.5: '-',
                                  0.54: '-',
                                  0.665: {'A1': {'a': '-', 'b': '+'}},
                                  0.705: '-',
                                  0.75: '-',
                                  0.835: '-',
                                  0.875: '-',
                                  1.0: '-',
                                  1.085: '-',

In [19]:
def predict(inst,tree):
    #This function is used to predict for any input variable 
    
    #Recursively we go through the tree that we built earlier

    for nodes in tree.keys():        
        
        value = inst[nodes]
        tree = tree[nodes][value]
        prediction = 0
            
        if type(tree) is dict:
            prediction = predict(inst, tree)
        else:
            prediction = tree
            break;                            
        
    return prediction

In [80]:
def get_predicted_labels(df):
    predict_labels = []
    for i in range(len(df.index)):
        inst = df.iloc[i]
        prediction = predict(inst, tree)
        predict_labels.append(prediction)
    original_labels = df['A16'].values
    predict_labels = np.array(predict_labels)
    return predict_labels, original_labels

In [72]:
def accuracy(orig, pred):
    num = len(pred)
    if(num != len(pred)):
        print('Error!! Num of labels are not equal.')
        return
    match = 0
    for i in range(len(orig)):
        o_label = orig[i]
        p_label = pred[i]
        if(o_label == p_label):
            match += 1
    print('***************\nAccuracy: ' + str(float(match)/num) + '\n***************')

In [82]:
df_test= pd.read_csv("test.txt",sep = '\t', header = None, names = attributes.keys())
# df_test
fill_missing_values(df_test)
df_test
# orig,pred = get_predicted_labels(df_test)
# print(len(orig), len(pred))

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,33.17,1.040,u,g,r,h,6.500,t,f,0,t,g,164,31285,+
1,a,22.92,11.585,u,g,cc,v,0.040,t,f,0,f,g,80,1349,+
2,b,54.42,0.500,y,p,k,h,3.960,t,f,0,f,g,180,314,+
3,b,42.5,4.915,y,p,w,v,3.165,t,f,0,t,g,52,1442,+
4,b,22.08,0.830,u,g,c,h,2.165,f,f,0,t,g,128,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,b,32.33,3.500,u,g,k,v,0.500,f,f,0,t,g,232,0,-
130,b,?,0.040,y,p,d,v,4.250,f,f,0,t,g,460,0,-
131,b,47.83,4.165,u,g,x,bb,0.085,f,f,0,t,g,520,0,-
132,b,20,1.250,y,p,k,v,0.125,f,f,0,f,g,140,4,-


In [79]:
accuracy_score = accuracy(original_labels, predict_labels)
accuracy_score

***************
Accuracy: 1.0
***************


In [86]:
# def find_entropy(df):
#     Class = df.keys()[-1]   #To make the code generic, changing target variable class name
#     entropy = 0
#     values = df[Class].unique()
#     for value in values:
#         fraction = df[Class].value_counts()[value]/len(df[Class])
#         entropy += -fraction*np.log2(fraction)
#     return entropy
def entropy_continuous(df,data_portion):
    data_portion_count = len(data_portion)
    output = {}
    entropy = 0
    for x,y in data_portion:
        if y in output:
            output[y].append((x,y))
        else:
            output[y] = [(x,y)]
    pos_labels_count = len(output.get('+'))
    neg_labels_count = data_portion_count - pos_labels_count
    pos_fraction = pos_labels_count/(data_portion_count+eps)
    pos_entropy = -pos_fraction*log(pos_fraction+eps)
    neg_fraction = neg_labels_count/(data_portion_count+eps)
    neg_entropy = -neg_fraction*log(neg_fraction+eps)
    entropy = pos_entropy + neg_entropy
    return entropy
df['A2']
df['A16']
zipped = zip(df['A2'],df['A16'])
zipped = list(zipped)
# print(str(zipped))
# Using sorted and lambda
result = sorted(zipped, key = lambda x: x[0])
result
row=0
curr_row=0
for i in range(len(result)):
    entropy = 0
    if(i!=0 and result[i][0]!=result[i-1][0] and i<len(result)):
        mid_value = (float(result[i][0])+float(result[i-1][0]))/2.0
#         print(mid_value)
        left_part = result[0:i]
        left_part_count = len(left_part) 
        entropy_left = entropy_continuous(df,left_part)
        left_fraction = len(left_part)/len(df)
        weighted_entropy_left = left_fraction*entropy_left
        entropy+=weighted_entropy_left
#         print("left: ", weighted_entropy_left)
#         print(entropy_left)
#         print(type(result))
#         print(left_part_count)
        right_part = result[i:]
        right_part_count = len(right_part) 
        entropy_right = entropy_continuous(df,right_part)
        right_fraction = len(right_part)/len(df)
        weighted_entropy_right = right_fraction*entropy_right
        entropy+= weighted_entropy_right
        print("Total entropy:", entropy)
#         print(right_part_count)
    else:
#         j=0
        print("Do nothing:", i)
# result
# Output = {}
# for x, y in result:
#     if y in Output:
#         Output[y].append((x, y))
#     else:
#         Output[y] = [(x, y)]
  
# # Printing Output
# print(len(Output.get('+')))

Do nothing: 0
Do nothing: 1
Total entropy: 0.9876080291683985
Total entropy: 0.9874683088888969
Do nothing: 4
Total entropy: 0.9858380798307065
Total entropy: 0.9847365554761287
Total entropy: 0.9835304548029735
Do nothing: 8
Total entropy: 0.9809089518596746
Total entropy: 0.9795232702406126
Total entropy: 0.9780998384403908
Do nothing: 12
Total entropy: 0.9751621364694123
Total entropy: 0.9736557633255575
Total entropy: 0.9721282010260093
Total entropy: 0.9705815883131358
Do nothing: 17
Total entropy: 0.9674378229973432
Total entropy: 0.9725304179942422
Do nothing: 20
Do nothing: 21
Total entropy: 0.9681677235046194
Total entropy: 0.9723073044512245
Total entropy: 0.9709484312975485
Do nothing: 25
Do nothing: 26
Total entropy: 0.9667410273079293
Do nothing: 28
Total entropy: 0.9638394152474749
Total entropy: 0.9623625742177208
Do nothing: 31
Total entropy: 0.9650425662059804
Total entropy: 0.9685561172449335
Do nothing: 34
Total entropy: 0.9659065798770924
Total entropy: 0.9690924775

TypeError: object of type 'NoneType' has no len()

In [14]:
# attribute = 'A9'
# target_variables = df.A16.unique()  #This gives all 'Yes' and 'No'
# variables = df[attribute].unique()    #This gives different features in that attribute (like 'Sweet')
# entropy_attribute = 0
# for variable in variables:
#     entropy_each_feature = 0
#     for target_variable in target_variables:
#         num = len(df[attribute][df[attribute]==variable][df.A16 ==target_variable]) #numerator
#         den = len(df[attribute][df[attribute]==variable])  #denominator
#         fraction = num/(den+eps)  #pi
#         entropy_each_feature += -fraction*log(fraction+eps) #This calculates entropy for one feature like 'Sweet'
#     fraction2 = den/len(df)
#     entropy_attribute += -fraction2*entropy_each_feature   #Sums up all the entropy ETaste
    
# abs(entropy_attribute)
# entropy_node-abs(entropy_attribute)

0.6343316941420678